# Combinar xlsx de export comments
Script para juntar los archivos de export comments en uno solo, incluyendo el id del post al que corresponde. 

In [2]:
# Requerimientos
# pip install openpyxl

# Importar
import glob
import openpyxl
from pathlib import Path
import pandas as pd

In [3]:
# Path a la carpeta con archivos xlsx a procesar
path_dir = 'virgiliomendozaa'
# Nombre del archivo de salida
file_name = 'DS Virgilio Mendoza.xlsx'

## Entendiendo la estructura del archivo

In [4]:
# Obtener los archivos xlsx
xlsx_files = [path for path in Path(path_dir).rglob('*.xlsx')]
# Para purebas seleccionar solo el primer archivo
xlsx_file = xlsx_files[0]
# Instanciar objeto y activar unica hoja del xlsx
wb_obj = openpyxl.load_workbook(xlsx_file)
sheet = wb_obj.active

In [5]:
# El id del post está en el link que se ubica en B2
print(sheet['B2'].value.split('/')[-1].split('?')[0])

327215485429583


In [6]:
# Desplegar los primeros 3 registros del xlsx
# Se muestran desde el renglón 7, antes son espacios en blanco y link
for row in sheet.iter_rows(min_row=7, max_row=10, min_col=3, max_col=9):
    for cell in row:
        print(cell.value, end=" ")
    print()

Oscar Fco Ramírez ID: 598705107 2021-05-06 16:15:53 2 None VAMOS CON TODO ....  #VAMOSVERDES  !!!!   💚✌💚✌💚✌   view comment 
Hiram Rigoberto Martinez Nuñez ID: 100036892331855 2021-05-06 16:16:33 0 None vamos con todo
[PHOTO] https://external.fiqt2-1.fna.fbcdn.net/safe_image.php?d=AQEURsv827fPrnrS&w=396&h=221&url=https%3A%2F%2Fmedia1.tenor.co%2Fimages%2Fd3683f994db8fef6b22d284fe64f301f%2Ftenor.gif%3Fitemid%3D15989998&sx=0&sy=0&sw=640&sh=357&ext=emg0&_nc_oe=6ef07&_nc_sid=632586&ccb=3-5&_nc_hash=AQEtegQIJW5I3FVQ view comment 
Elva Larios ID: 100023679033955 2021-05-06 16:19:29 0 None Tu experiencia y el apoyo que has brindado a tu gente, te respaldan, Virgilio Mendoza próximo gobernador de Colima VERDE 💚💚💚💚💚💚💚 view comment 
Gloria Andrade ID: 100015002491697 2021-05-06 16:20:39 2 None 
[PHOTO] https://scontent.fiqt2-1.fna.fbcdn.net/v/t1.6435-9/s600x600/182872655_1109475749562498_4223984717201769315_n.jpg?_nc_cat=111&ccb=1-5&_nc_sid=dbeb18&_nc_ohc=GWIi48M1o4wAX_kyh6X&_nc_ht=scontent.fiqt2-

In [7]:
# Verificar los renglones (el ultimo renglón no tiene valores) y columnas
print(sheet.max_row, sheet.max_column)

135 9


In [8]:
# Obtener link en una celda
comment_link = sheet['I7'].hyperlink.target
print(comment_link)
# Limpiar link para sólo conservar el id
comment_id = comment_link.split('/')[-1]
print(comment_id)

https://www.exportcomments.com/done/a483fb4b-3949-4266-99c4-ab289e5a0710/preview/327217365429395
327217365429395


## Juntando archivos
Consideraciones
- No tener abierto los archivos
- Solo tener archivos exportado por export comments en la carpeta

In [9]:
def get_post_id(sheet):
    return [sheet['B2'].value.split('/')[-1].split('?')[0]]

In [10]:
def get_data_row(row):
    user_name = row[0].value
    profile_id = row[1].value.split(' ')[-1]
    date_time = row[2].value
    likes = row[3].value
    comments = row[5].value
    # Obtener link del comentario
    comment_link = row[6].hyperlink.target
    # Limpiar y guaardar sólo id
    comment_id = comment_link.split('/')[-1]
    return [user_name, profile_id, date_time, likes, comments, comment_id]

In [11]:
# Nombres de columnas
columns = ['post_id','user_name','profile_id','date_time','likes','comments','comment_id']
data = []

# Obteniendo todos los archivos xlsx
xlsx_files = [path for path in Path(path_dir).rglob('*.xlsx')]
# Generando objetos para cada archivo
wb_objects = [openpyxl.load_workbook(wb) for wb in xlsx_files]

# Procesando todos los objetos xlsx
for wb_obj in wb_objects:
    sheet = wb_obj.active
    post_id = get_post_id(sheet)
    # El maximo renglón a procesar es -1 ya que el último es de valores nulos
    max_row = sheet.max_row - 1
    # Procesado de todos los valores con función get_data_row y get_post_id
    for row in sheet.iter_rows(min_row=7, max_row=max_row, min_col=3, max_col=9):
        comment_data = post_id + get_data_row(row)
        # El rengló se agrega a la lista
        data.append(comment_data)
        
# Con los datos y columnas se genera el dataframe
df = pd.DataFrame(data, columns=columns)

### Revisando datos del dataframe

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26431 entries, 0 to 26430
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   post_id     26431 non-null  object        
 1   user_name   26431 non-null  object        
 2   profile_id  26431 non-null  object        
 3   date_time   26431 non-null  datetime64[ns]
 4   likes       26431 non-null  int64         
 5   comments    26430 non-null  object        
 6   comment_id  26431 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.4+ MB


In [13]:
df.head(5)

,post_id,user_name,profile_id,date_time,likes,comments,comment_id
0,327215485429583,Oscar Fco Ramírez,598705107,2021-05-06 16:15:53,2,VAMOS CON TODO .... #VAMOSVERDES !!!! 💚✌💚✌...,327217365429395
1,327215485429583,Hiram Rigoberto Martinez Nuñez,100036892331855,2021-05-06 16:16:33,0,vamos con todo\n[PHOTO] https://external.fiqt2...,327217635429368
2,327215485429583,Elva Larios,100023679033955,2021-05-06 16:19:29,0,Tu experiencia y el apoyo que has brindado a t...,327218888762576
3,327215485429583,Gloria Andrade,100015002491697,2021-05-06 16:20:39,2,\n[PHOTO] https://scontent.fiqt2-1.fna.fbcdn.n...,327219572095841
4,327215485429583,Gloria Andrade,100015002491697,2021-05-06 16:20:47,2,\n[PHOTO] https://scontent.fiqt2-1.fna.fbcdn.n...,327219635429168


### Guardando en archivo

In [14]:
df.to_excel(file_name, sheet_name='Comments', header=True, index=False)

## Referencias

- https://www.marsja.se/your-guide-to-reading-excel-xlsx-files-in-python/
- https://stackoverflow.com/questions/29792134/how-we-can-use-iter-rows-in-python-openpyxl-package
- https://openpyxl.readthedocs.io/en/stable/datetime.html
- https://newbedev.com/extracting-hyperlinks-from-excel-xlsx-with-python
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html